# HW

Get statistic from MedHistory

## Prep Data

### Read Data

In [35]:
import os
import random
import math
import json
import time
import pandas as pd
import numpy as np
import copy as copy
import matplotlib.pyplot as plt
import translators as ts
import translators.server as tss
from translate import Translator
from pandas import Timestamp
from datetime import datetime
from os import getcwd
from os.path import join
%matplotlib inline


In [36]:
data_path_1 = os.path.join(
    os.getcwd().rstrip('src'), 'data', 'TaiwanDeathDetail2021.xlsx')
data_path_2 = os.path.join(
    os.getcwd().rstrip('src'), 'data', 'TaiwanDeathDetail20220611.xlsx')
data1 = pd.read_excel(data_path_1, sheet_name='TaiwanDeathDetail2021.csv')
data2 = pd.read_excel(data_path_2, sheet_name='TaiwanDeathDetail20220611.csv')
data1.head()

,公布日,案號,性別,年齡,慢性病史,活動接觸史,發病日,症狀,採檢日,住院/隔離日,確診日,死亡日
0,2022-12-19,4397,男,60多歲,有,無,2022-05-22 00:00:00,發燒,2022-05-22 00:00:00,2022-05-23 00:00:00,2022-05-24 00:00:00,2022-12-01 00:00:00
1,2022-12-15,10511,男,50多歲,有,有,2022-06-03 00:00:00,呼吸喘,2022-06-03 00:00:00,6/4住院，8月14日解除隔離,2022-06-05 00:00:00,2022-11-19 00:00:00
2,2022-11-09,8724,女,90多歲,有,有,2022-05-24 00:00:00,發燒、喉嚨痛,2022-05-31 00:00:00,5/30住院、\n7/26解除隔離,2022-06-01 00:00:00,2022-10-11 00:00:00
3,2022-10-25,12235,男,80多歲,有,接觸其他確診者,2022-06-08 00:00:00,食慾不振、昏睡,2022-06-08 00:00:00,6/8\n7/14解除隔離,2022-06-10 00:00:00,2022-10-22 00:00:00
4,2022-10-09,8748,女,70多歲,有,接觸其他確診者,--,--,2022-05-30 00:00:00,5/31住院\n9/17解除隔離,2022-06-01 00:00:00,2022-10-04 00:00:00


In [37]:
data2.head()

,日期,案號,性別,年齡,慢性病史,是否接種COVID19疫苗,發病日,症狀,採檢日,住院／隔離日,確診日,死亡日
0,2022-06-11,NaN,男,90多歲以上,慢性肺病,3劑,2022-05-18 00:00:00,發燒,NaN,NaN,2022-05-18 00:00:00,2022-06-06
1,2022-06-11,NaN,女,90多歲以上,高血壓等,無,2022-06-01 00:00:00,發燒、咳嗽、呼吸喘,NaN,NaN,2022-06-01 00:00:00,2022-06-07
2,2022-06-11,NaN,女,90多歲以上,心血管疾病,無,2022-05-30 00:00:00,咳嗽、呼吸困難、顫抖,NaN,NaN,2022-05-30 00:00:00,2022-06-06
3,2022-06-11,NaN,女,90多歲以上,心血管疾病等,1劑,調查中,調查中,NaN,NaN,2022-06-06 00:00:00,2022-06-07
4,2022-06-11,NaN,女,90多歲以上,神經系統疾病等,3劑,2022-05-20 00:00:00,呼吸困難、血氧下降、胸部X光片肺部 雙側浸潤,NaN,NaN,2022-05-22 00:00:00,2022-05-21


### Translate Column Names

In [38]:
columns_ts = ['Date', 'CaseID', 'Gender', 'Age', 'MedHistory', 'Vac', 'SickDate', 'Symptoms', 'SpecimenDate', 'QuarantineDate', 'ConfirmDate', 'DeceaseDate']
columns_dict = dict(zip(data1.columns, columns_ts))
print(columns_dict)
data1.rename(columns=columns_dict, inplace=True)
data1.head()

{'公布日': 'Date', '案號': 'CaseID', '性別': 'Gender', '年齡': 'Age', '慢性病史': 'MedHistory', '活動接觸史': 'Vac', '發病日': 'SickDate', '症狀': 'Symptoms', '採檢日': 'SpecimenDate', '住院/隔離日': 'QuarantineDate', '確診日': 'ConfirmDate', '死亡日': 'DeceaseDate'}


,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022-12-19,4397,男,60多歲,有,無,2022-05-22 00:00:00,發燒,2022-05-22 00:00:00,2022-05-23 00:00:00,2022-05-24 00:00:00,2022-12-01 00:00:00
1,2022-12-15,10511,男,50多歲,有,有,2022-06-03 00:00:00,呼吸喘,2022-06-03 00:00:00,6/4住院，8月14日解除隔離,2022-06-05 00:00:00,2022-11-19 00:00:00
2,2022-11-09,8724,女,90多歲,有,有,2022-05-24 00:00:00,發燒、喉嚨痛,2022-05-31 00:00:00,5/30住院、\n7/26解除隔離,2022-06-01 00:00:00,2022-10-11 00:00:00
3,2022-10-25,12235,男,80多歲,有,接觸其他確診者,2022-06-08 00:00:00,食慾不振、昏睡,2022-06-08 00:00:00,6/8\n7/14解除隔離,2022-06-10 00:00:00,2022-10-22 00:00:00
4,2022-10-09,8748,女,70多歲,有,接觸其他確診者,--,--,2022-05-30 00:00:00,5/31住院\n9/17解除隔離,2022-06-01 00:00:00,2022-10-04 00:00:00


In [39]:
columns_ts = ['Date', 'CaseID', 'Gender', 'Age', 'MedHistory', 'Vac', 'SickDate',
              'Symptoms', 'SpecimenDate', 'QuarantineDate', 'ConfirmDate', 'DeceaseDate']
columns_dict = dict(zip(data2.columns, columns_ts))
print(columns_dict)
data2.rename(columns=columns_dict, inplace=True)
data2.head()

{'日期': 'Date', '案號': 'CaseID', '性別': 'Gender', '年齡': 'Age', '慢性病史': 'MedHistory', '是否接種COVID19疫苗': 'Vac', '發病日': 'SickDate', '症狀': 'Symptoms', '採檢日': 'SpecimenDate', '住院／隔離日': 'QuarantineDate', '確診日': 'ConfirmDate', '死亡日': 'DeceaseDate'}


,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022-06-11,NaN,男,90多歲以上,慢性肺病,3劑,2022-05-18 00:00:00,發燒,NaN,NaN,2022-05-18 00:00:00,2022-06-06
1,2022-06-11,NaN,女,90多歲以上,高血壓等,無,2022-06-01 00:00:00,發燒、咳嗽、呼吸喘,NaN,NaN,2022-06-01 00:00:00,2022-06-07
2,2022-06-11,NaN,女,90多歲以上,心血管疾病,無,2022-05-30 00:00:00,咳嗽、呼吸困難、顫抖,NaN,NaN,2022-05-30 00:00:00,2022-06-06
3,2022-06-11,NaN,女,90多歲以上,心血管疾病等,1劑,調查中,調查中,NaN,NaN,2022-06-06 00:00:00,2022-06-07
4,2022-06-11,NaN,女,90多歲以上,神經系統疾病等,3劑,2022-05-20 00:00:00,呼吸困難、血氧下降、胸部X光片肺部 雙側浸潤,NaN,NaN,2022-05-22 00:00:00,2022-05-21


### Translate Column 'Gender'

In [40]:
gender_dict = {'男': 'M', '女': 'F'}
data1['Gender'] = data1['Gender'].map(gender_dict)
data1.head()

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022-12-19,4397,M,60多歲,有,無,2022-05-22 00:00:00,發燒,2022-05-22 00:00:00,2022-05-23 00:00:00,2022-05-24 00:00:00,2022-12-01 00:00:00
1,2022-12-15,10511,M,50多歲,有,有,2022-06-03 00:00:00,呼吸喘,2022-06-03 00:00:00,6/4住院，8月14日解除隔離,2022-06-05 00:00:00,2022-11-19 00:00:00
2,2022-11-09,8724,F,90多歲,有,有,2022-05-24 00:00:00,發燒、喉嚨痛,2022-05-31 00:00:00,5/30住院、\n7/26解除隔離,2022-06-01 00:00:00,2022-10-11 00:00:00
3,2022-10-25,12235,M,80多歲,有,接觸其他確診者,2022-06-08 00:00:00,食慾不振、昏睡,2022-06-08 00:00:00,6/8\n7/14解除隔離,2022-06-10 00:00:00,2022-10-22 00:00:00
4,2022-10-09,8748,F,70多歲,有,接觸其他確診者,--,--,2022-05-30 00:00:00,5/31住院\n9/17解除隔離,2022-06-01 00:00:00,2022-10-04 00:00:00


In [41]:
data2['Gender'] = data2['Gender'].map(gender_dict)
data2.head()

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022-06-11,NaN,M,90多歲以上,慢性肺病,3劑,2022-05-18 00:00:00,發燒,NaN,NaN,2022-05-18 00:00:00,2022-06-06
1,2022-06-11,NaN,F,90多歲以上,高血壓等,無,2022-06-01 00:00:00,發燒、咳嗽、呼吸喘,NaN,NaN,2022-06-01 00:00:00,2022-06-07
2,2022-06-11,NaN,F,90多歲以上,心血管疾病,無,2022-05-30 00:00:00,咳嗽、呼吸困難、顫抖,NaN,NaN,2022-05-30 00:00:00,2022-06-06
3,2022-06-11,NaN,F,90多歲以上,心血管疾病等,1劑,調查中,調查中,NaN,NaN,2022-06-06 00:00:00,2022-06-07
4,2022-06-11,NaN,F,90多歲以上,神經系統疾病等,3劑,2022-05-20 00:00:00,呼吸困難、血氧下降、胸部X光片肺部 雙側浸潤,NaN,NaN,2022-05-22 00:00:00,2022-05-21


### Translate Column 'Age'

In [42]:
ages = data1['Age'].unique().tolist()
ages.sort()
print(ages)
ages_ts = [100.5, 30.5, 40.5, 50.5, 60.5, 70.5, 80.5, 90.5]
ages_dict = dict(zip(ages, ages_ts))
print(ages_dict)
data1['Age'] = data1['Age'].map(ages_dict)
data1.head()

['100多歲', '30多歲', '40多歲', '50多歲', '60多歲', '70多歲', '80多歲', '90多歲']
{'100多歲': 100.5, '30多歲': 30.5, '40多歲': 40.5, '50多歲': 50.5, '60多歲': 60.5, '70多歲': 70.5, '80多歲': 80.5, '90多歲': 90.5}


,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022-12-19,4397,M,60.5,有,無,2022-05-22 00:00:00,發燒,2022-05-22 00:00:00,2022-05-23 00:00:00,2022-05-24 00:00:00,2022-12-01 00:00:00
1,2022-12-15,10511,M,50.5,有,有,2022-06-03 00:00:00,呼吸喘,2022-06-03 00:00:00,6/4住院，8月14日解除隔離,2022-06-05 00:00:00,2022-11-19 00:00:00
2,2022-11-09,8724,F,90.5,有,有,2022-05-24 00:00:00,發燒、喉嚨痛,2022-05-31 00:00:00,5/30住院、\n7/26解除隔離,2022-06-01 00:00:00,2022-10-11 00:00:00
3,2022-10-25,12235,M,80.5,有,接觸其他確診者,2022-06-08 00:00:00,食慾不振、昏睡,2022-06-08 00:00:00,6/8\n7/14解除隔離,2022-06-10 00:00:00,2022-10-22 00:00:00
4,2022-10-09,8748,F,70.5,有,接觸其他確診者,--,--,2022-05-30 00:00:00,5/31住院\n9/17解除隔離,2022-06-01 00:00:00,2022-10-04 00:00:00


In [43]:
ages = data2['Age'].unique().tolist()
print(ages)
ages_ts = [90.5, 80.5, 70.5, 60.5, 50.5, 40.5, 30.5, 90.5, 20.5, 10.5, 4.5, 9.5, np.nan, 102, 101, 100, 98, 97, 95, 94, 96, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 64, 62, 61, 60, 55, 48, 38, 22, 4, 1, 90.5]
ages_dict = dict(zip(ages, ages_ts))
print(ages_dict)
data2['Age'] = data2['Age'].map(ages_dict)
data2.head()

['90多歲以上', '80多歲', '70多歲', '60多歲', '50多歲', '40多歲', '30多歲', '90歲以上', '20多歲', '10多歲', '未滿5歲', '未滿10歲', nan, '102歲', '101歲', '100歲', '98歲', '97歲', '95歲', '94歲', '93歲', '92歲', '91歲', '90歲', '89歲', '88歲', '87歲', '86歲', '85歲', '84歲', '83歲', '82歲', '81歲', '80歲', '79歲', '78歲', '77歲', '76歲', '75歲', '74歲', '73歲', '72歲', '71歲', '70歲', '69歲', '68歲', '67歲', '64歲', '62歲', '61歲', '60歲', '55歲', '48歲', '38歲', '22歲', '4歲', '1歲', '90多歲']
{'90多歲以上': 90.5, '80多歲': 80.5, '70多歲': 70.5, '60多歲': 60.5, '50多歲': 50.5, '40多歲': 40.5, '30多歲': 30.5, '90歲以上': 90.5, '20多歲': 20.5, '10多歲': 10.5, '未滿5歲': 4.5, '未滿10歲': 9.5, nan: nan, '102歲': 102, '101歲': 101, '100歲': 100, '98歲': 98, '97歲': 97, '95歲': 95, '94歲': 94, '93歲': 96, '92歲': 92, '91歲': 91, '90歲': 90, '89歲': 89, '88歲': 88, '87歲': 87, '86歲': 86, '85歲': 85, '84歲': 84, '83歲': 83, '82歲': 82, '81歲': 81, '80歲': 80, '79歲': 79, '78歲': 78, '77歲': 77, '76歲': 76, '75歲': 75, '74歲': 74, '73歲': 73, '72歲': 72, '71歲': 71, '70歲': 70, '69歲': 69, '68歲': 68, '67歲': 67, '64歲': 64, '62歲'

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022-06-11,NaN,M,90.5,慢性肺病,3劑,2022-05-18 00:00:00,發燒,NaN,NaN,2022-05-18 00:00:00,2022-06-06
1,2022-06-11,NaN,F,90.5,高血壓等,無,2022-06-01 00:00:00,發燒、咳嗽、呼吸喘,NaN,NaN,2022-06-01 00:00:00,2022-06-07
2,2022-06-11,NaN,F,90.5,心血管疾病,無,2022-05-30 00:00:00,咳嗽、呼吸困難、顫抖,NaN,NaN,2022-05-30 00:00:00,2022-06-06
3,2022-06-11,NaN,F,90.5,心血管疾病等,1劑,調查中,調查中,NaN,NaN,2022-06-06 00:00:00,2022-06-07
4,2022-06-11,NaN,F,90.5,神經系統疾病等,3劑,2022-05-20 00:00:00,呼吸困難、血氧下降、胸部X光片肺部 雙側浸潤,NaN,NaN,2022-05-22 00:00:00,2022-05-21


### Translate Column 'Vac'

In [44]:
columns = data1['Vac'].unique().tolist()
print(columns)
columns_ts = []
for element in columns:
    if element == '有':
        columns_ts.append('Y')
    elif element == '無':
        columns_ts.append('N')
    else:
        columns_ts.append(np.nan)
print(columns_ts)
columns_dict = dict(zip(columns, columns_ts))
print(columns_dict)
data1['Vac'] = data1['Vac'].map(columns_dict)
data1.head()

['無', '有', '接觸其他確診者', '萬華活動史', '--', '8月6日自越南入境', '萬華區活動史', nan, '相關活動接觸史', '接觸其他確診案', '調查中', '接觸其他確診個案', '4/11境外移入', '菲律賓回台', '緬甸回台', '境外移入（英國變種病毒）']
['N', 'Y', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
{'無': 'N', '有': 'Y', '接觸其他確診者': nan, '萬華活動史': nan, '--': nan, '8月6日自越南入境': nan, '萬華區活動史': nan, nan: nan, '相關活動接觸史': nan, '接觸其他確診案': nan, '調查中': nan, '接觸其他確診個案': nan, '4/11境外移入': nan, '菲律賓回台': nan, '緬甸回台': nan, '境外移入（英國變種病毒）': nan}


,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022-12-19,4397,M,60.5,有,N,2022-05-22 00:00:00,發燒,2022-05-22 00:00:00,2022-05-23 00:00:00,2022-05-24 00:00:00,2022-12-01 00:00:00
1,2022-12-15,10511,M,50.5,有,Y,2022-06-03 00:00:00,呼吸喘,2022-06-03 00:00:00,6/4住院，8月14日解除隔離,2022-06-05 00:00:00,2022-11-19 00:00:00
2,2022-11-09,8724,F,90.5,有,Y,2022-05-24 00:00:00,發燒、喉嚨痛,2022-05-31 00:00:00,5/30住院、\n7/26解除隔離,2022-06-01 00:00:00,2022-10-11 00:00:00
3,2022-10-25,12235,M,80.5,有,NaN,2022-06-08 00:00:00,食慾不振、昏睡,2022-06-08 00:00:00,6/8\n7/14解除隔離,2022-06-10 00:00:00,2022-10-22 00:00:00
4,2022-10-09,8748,F,70.5,有,NaN,--,--,2022-05-30 00:00:00,5/31住院\n9/17解除隔離,2022-06-01 00:00:00,2022-10-04 00:00:00


In [45]:
columns = data2['Vac'].unique().tolist()
print(columns)
columns_ts = [3, 0, 1, 2, 4, np.nan, np.nan]
columns_dict = dict(zip(columns, columns_ts))
print(columns_dict)
data2['Vac'] = data2['Vac'].map(columns_dict)
data2.head()


['3劑', '無', '1劑', '2劑', '4劑', '調查中', nan]
{'3劑': 3, '無': 0, '1劑': 1, '2劑': 2, '4劑': 4, '調查中': nan, nan: nan}


,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022-06-11,NaN,M,90.5,慢性肺病,3.0,2022-05-18 00:00:00,發燒,NaN,NaN,2022-05-18 00:00:00,2022-06-06
1,2022-06-11,NaN,F,90.5,高血壓等,0.0,2022-06-01 00:00:00,發燒、咳嗽、呼吸喘,NaN,NaN,2022-06-01 00:00:00,2022-06-07
2,2022-06-11,NaN,F,90.5,心血管疾病,0.0,2022-05-30 00:00:00,咳嗽、呼吸困難、顫抖,NaN,NaN,2022-05-30 00:00:00,2022-06-06
3,2022-06-11,NaN,F,90.5,心血管疾病等,1.0,調查中,調查中,NaN,NaN,2022-06-06 00:00:00,2022-06-07
4,2022-06-11,NaN,F,90.5,神經系統疾病等,3.0,2022-05-20 00:00:00,呼吸困難、血氧下降、胸部X光片肺部 雙側浸潤,NaN,NaN,2022-05-22 00:00:00,2022-05-21


### Convert Column 'Date'

In [46]:
data1['Date'] = data1['Date'].dt.strftime('%Y/%m/%d')
data1.head()


,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022/12/19,4397,M,60.5,有,N,2022-05-22 00:00:00,發燒,2022-05-22 00:00:00,2022-05-23 00:00:00,2022-05-24 00:00:00,2022-12-01 00:00:00
1,2022/12/15,10511,M,50.5,有,Y,2022-06-03 00:00:00,呼吸喘,2022-06-03 00:00:00,6/4住院，8月14日解除隔離,2022-06-05 00:00:00,2022-11-19 00:00:00
2,2022/11/09,8724,F,90.5,有,Y,2022-05-24 00:00:00,發燒、喉嚨痛,2022-05-31 00:00:00,5/30住院、\n7/26解除隔離,2022-06-01 00:00:00,2022-10-11 00:00:00
3,2022/10/25,12235,M,80.5,有,NaN,2022-06-08 00:00:00,食慾不振、昏睡,2022-06-08 00:00:00,6/8\n7/14解除隔離,2022-06-10 00:00:00,2022-10-22 00:00:00
4,2022/10/09,8748,F,70.5,有,NaN,--,--,2022-05-30 00:00:00,5/31住院\n9/17解除隔離,2022-06-01 00:00:00,2022-10-04 00:00:00


In [47]:
data2['Date'] = data2['Date'].dt.strftime('%Y/%m/%d')
data2.head()

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022/06/11,NaN,M,90.5,慢性肺病,3.0,2022-05-18 00:00:00,發燒,NaN,NaN,2022-05-18 00:00:00,2022-06-06
1,2022/06/11,NaN,F,90.5,高血壓等,0.0,2022-06-01 00:00:00,發燒、咳嗽、呼吸喘,NaN,NaN,2022-06-01 00:00:00,2022-06-07
2,2022/06/11,NaN,F,90.5,心血管疾病,0.0,2022-05-30 00:00:00,咳嗽、呼吸困難、顫抖,NaN,NaN,2022-05-30 00:00:00,2022-06-06
3,2022/06/11,NaN,F,90.5,心血管疾病等,1.0,調查中,調查中,NaN,NaN,2022-06-06 00:00:00,2022-06-07
4,2022/06/11,NaN,F,90.5,神經系統疾病等,3.0,2022-05-20 00:00:00,呼吸困難、血氧下降、胸部X光片肺部 雙側浸潤,NaN,NaN,2022-05-22 00:00:00,2022-05-21


### Convert Column 'SickDate'

In [48]:
sds = data1['SickDate'].unique().tolist()
print(sds)
sds_ts = []
for element in sds:
    if element == '無症狀' or element == '調查中' or element == '--':
        sds_ts.append(np.nan)
    else:
        sds_ts.append(element)
# print(sds_ts)
sds_dict = dict(zip(sds, sds_ts))
# print(sds_dict)
data1['SickDate'] = data1['SickDate'].map(sds_dict)
data1.head()

[datetime.datetime(2022, 5, 22, 0, 0), datetime.datetime(2022, 6, 3, 0, 0), datetime.datetime(2022, 5, 24, 0, 0), datetime.datetime(2022, 6, 8, 0, 0), '--', datetime.datetime(2022, 6, 30, 0, 0), datetime.datetime(2022, 8, 10, 0, 0), datetime.datetime(2022, 5, 19, 0, 0), datetime.datetime(2022, 6, 19, 0, 0), datetime.datetime(2022, 8, 17, 0, 0), datetime.datetime(2022, 7, 9, 0, 0), datetime.datetime(2022, 9, 7, 0, 0), datetime.datetime(2022, 5, 21, 0, 0), datetime.datetime(2022, 5, 30, 0, 0), datetime.datetime(2022, 8, 26, 0, 0), datetime.datetime(2022, 6, 26, 0, 0), datetime.datetime(2022, 6, 27, 0, 0), datetime.datetime(2022, 6, 23, 0, 0), nan, datetime.datetime(2022, 7, 27, 0, 0), datetime.datetime(2022, 5, 25, 0, 0), datetime.datetime(2022, 6, 1, 0, 0), datetime.datetime(2022, 6, 2, 0, 0), datetime.datetime(2022, 6, 13, 0, 0), datetime.datetime(2022, 5, 17, 0, 0), datetime.datetime(2022, 5, 28, 0, 0), datetime.datetime(2022, 6, 17, 0, 0), datetime.datetime(2022, 6, 14, 0, 0), dateti

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022/12/19,4397,M,60.5,有,N,2022-05-22,發燒,2022-05-22 00:00:00,2022-05-23 00:00:00,2022-05-24 00:00:00,2022-12-01 00:00:00
1,2022/12/15,10511,M,50.5,有,Y,2022-06-03,呼吸喘,2022-06-03 00:00:00,6/4住院，8月14日解除隔離,2022-06-05 00:00:00,2022-11-19 00:00:00
2,2022/11/09,8724,F,90.5,有,Y,2022-05-24,發燒、喉嚨痛,2022-05-31 00:00:00,5/30住院、\n7/26解除隔離,2022-06-01 00:00:00,2022-10-11 00:00:00
3,2022/10/25,12235,M,80.5,有,NaN,2022-06-08,食慾不振、昏睡,2022-06-08 00:00:00,6/8\n7/14解除隔離,2022-06-10 00:00:00,2022-10-22 00:00:00
4,2022/10/09,8748,F,70.5,有,NaN,NaT,--,2022-05-30 00:00:00,5/31住院\n9/17解除隔離,2022-06-01 00:00:00,2022-10-04 00:00:00


In [49]:
data1['SickDate'] = data1['SickDate'].dt.strftime('%Y/%m/%d')
data1.head()

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022/12/19,4397,M,60.5,有,N,2022/05/22,發燒,2022-05-22 00:00:00,2022-05-23 00:00:00,2022-05-24 00:00:00,2022-12-01 00:00:00
1,2022/12/15,10511,M,50.5,有,Y,2022/06/03,呼吸喘,2022-06-03 00:00:00,6/4住院，8月14日解除隔離,2022-06-05 00:00:00,2022-11-19 00:00:00
2,2022/11/09,8724,F,90.5,有,Y,2022/05/24,發燒、喉嚨痛,2022-05-31 00:00:00,5/30住院、\n7/26解除隔離,2022-06-01 00:00:00,2022-10-11 00:00:00
3,2022/10/25,12235,M,80.5,有,NaN,2022/06/08,食慾不振、昏睡,2022-06-08 00:00:00,6/8\n7/14解除隔離,2022-06-10 00:00:00,2022-10-22 00:00:00
4,2022/10/09,8748,F,70.5,有,NaN,NaN,--,2022-05-30 00:00:00,5/31住院\n9/17解除隔離,2022-06-01 00:00:00,2022-10-04 00:00:00


In [50]:
sds = data2['SickDate'].unique().tolist()
print(sds)
sds_ts = []
for element in sds:
    if type(element) == str:
        sds_ts.append(np.nan)
    else:
        sds_ts.append(element)
# print(sds_ts)
sds_dict = dict(zip(sds, sds_ts))
# print(sds_dict)
data2['SickDate'] = data2['SickDate'].map(sds_dict)
data2.head()

[datetime.datetime(2022, 5, 18, 0, 0), datetime.datetime(2022, 6, 1, 0, 0), datetime.datetime(2022, 5, 30, 0, 0), '調查中', datetime.datetime(2022, 5, 20, 0, 0), '機構住民，篩檢確診', datetime.datetime(2022, 5, 24, 0, 0), '長期臥床', datetime.datetime(2022, 5, 28, 0, 0), datetime.datetime(2022, 5, 31, 0, 0), datetime.datetime(2022, 5, 22, 0, 0), datetime.datetime(2022, 6, 4, 0, 0), datetime.datetime(2022, 5, 23, 0, 0), datetime.datetime(2022, 5, 13, 0, 0), datetime.datetime(2022, 6, 5, 0, 0), datetime.datetime(2022, 5, 29, 0, 0), datetime.datetime(2022, 6, 7, 0, 0), datetime.datetime(2022, 6, 2, 0, 0), datetime.datetime(2022, 5, 27, 0, 0), datetime.datetime(2022, 6, 6, 0, 0), datetime.datetime(2022, 5, 14, 0, 0), datetime.datetime(2022, 5, 2, 0, 0), datetime.datetime(2022, 5, 19, 0, 0), datetime.datetime(2022, 5, 26, 0, 0), datetime.datetime(2022, 5, 12, 0, 0), '到院前無生命徵象', datetime.datetime(2022, 6, 3, 0, 0), datetime.datetime(2022, 5, 17, 0, 0), datetime.datetime(2022, 5, 15, 0, 0), datetime.datetime

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022/06/11,NaN,M,90.5,慢性肺病,3.0,2022-05-18,發燒,NaN,NaN,2022-05-18 00:00:00,2022-06-06
1,2022/06/11,NaN,F,90.5,高血壓等,0.0,2022-06-01,發燒、咳嗽、呼吸喘,NaN,NaN,2022-06-01 00:00:00,2022-06-07
2,2022/06/11,NaN,F,90.5,心血管疾病,0.0,2022-05-30,咳嗽、呼吸困難、顫抖,NaN,NaN,2022-05-30 00:00:00,2022-06-06
3,2022/06/11,NaN,F,90.5,心血管疾病等,1.0,NaT,調查中,NaN,NaN,2022-06-06 00:00:00,2022-06-07
4,2022/06/11,NaN,F,90.5,神經系統疾病等,3.0,2022-05-20,呼吸困難、血氧下降、胸部X光片肺部 雙側浸潤,NaN,NaN,2022-05-22 00:00:00,2022-05-21


In [51]:
data2['SickDate'] = data2['SickDate'].dt.strftime('%Y/%m/%d')
data2.head()

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022/06/11,NaN,M,90.5,慢性肺病,3.0,2022/05/18,發燒,NaN,NaN,2022-05-18 00:00:00,2022-06-06
1,2022/06/11,NaN,F,90.5,高血壓等,0.0,2022/06/01,發燒、咳嗽、呼吸喘,NaN,NaN,2022-06-01 00:00:00,2022-06-07
2,2022/06/11,NaN,F,90.5,心血管疾病,0.0,2022/05/30,咳嗽、呼吸困難、顫抖,NaN,NaN,2022-05-30 00:00:00,2022-06-06
3,2022/06/11,NaN,F,90.5,心血管疾病等,1.0,NaN,調查中,NaN,NaN,2022-06-06 00:00:00,2022-06-07
4,2022/06/11,NaN,F,90.5,神經系統疾病等,3.0,2022/05/20,呼吸困難、血氧下降、胸部X光片肺部 雙側浸潤,NaN,NaN,2022-05-22 00:00:00,2022-05-21


### Convert Column 'SpecimenDate'

In [54]:
sds = data1['SpecimenDate'].unique().tolist()
print(sds)

# data1['SpecimenDate'] = data1['SpecimenDate'].dt.strftime('%Y/%m/%d')
# data1.head()

[datetime.datetime(2022, 5, 22, 0, 0), datetime.datetime(2022, 6, 3, 0, 0), datetime.datetime(2022, 5, 31, 0, 0), datetime.datetime(2022, 6, 8, 0, 0), datetime.datetime(2022, 5, 30, 0, 0), datetime.datetime(2022, 6, 30, 0, 0), datetime.datetime(2022, 8, 10, 0, 0), datetime.datetime(2022, 6, 19, 0, 0), datetime.datetime(2022, 8, 18, 0, 0), datetime.datetime(2022, 7, 28, 0, 0), datetime.datetime(2022, 9, 7, 0, 0), datetime.datetime(2022, 5, 27, 0, 0), datetime.datetime(2022, 8, 28, 0, 0), datetime.datetime(2022, 8, 27, 0, 0), datetime.datetime(2022, 6, 29, 0, 0), datetime.datetime(2022, 6, 27, 0, 0), datetime.datetime(2022, 6, 24, 0, 0), datetime.datetime(2022, 8, 2, 0, 0), datetime.datetime(2022, 8, 5, 0, 0), datetime.datetime(2022, 5, 28, 0, 0), datetime.datetime(2022, 7, 16, 0, 0), datetime.datetime(2022, 6, 1, 0, 0), datetime.datetime(2022, 6, 2, 0, 0), datetime.datetime(2022, 6, 6, 0, 0), datetime.datetime(2022, 6, 13, 0, 0), datetime.datetime(2022, 6, 14, 0, 0), datetime.datetime(2